In [1]:
import pygame, sys, pymunk


def String_legs(space, index_0, index_1):
    joint = pymunk.PinJoint(balls[index_0].body,balls[index_1].body ,(0,0),(0,0))
    #motor = pymunk.SimpleMotor( balls[index_0].body, balls[index_1].body, 4.0)
    space.add(joint )
    return joint
    
    
# def jmotors(space,index_0,index_1): 
#     motor = pymunk.SimpleMotor( balls[index_0].body, balls[index_1].body, relangvel)
#     shape = pymunk.Circle()
#     space.add(motor)
#     return motor

def draw_leg(joints):
    for i in range(0,len(joints)):
        pygame.draw.line(screen, (0 ,0 ,0) ,
                         (int(balls[i].body.position.x),
                          int(balls[i].body.position.y)) , 
                         (int(balls[i+1].body.position.x),
                          int(balls[i+1].body.position.y)),5)
        

def falling_ball(space , pos_x, pos_y):
    body= pymunk.Body(1,10,body_type=pymunk.Body.DYNAMIC)
    body.position = (pos_x,pos_y)
    shape = pymunk.Circle(body,15)
    space.add(body,shape)
    return shape


def draw_ball(balls) :
    for i in balls:
        posx= int(i.body.position.x)
        posy =int(i.body.position.y)
        pygame.draw.circle(screen,(0,0,0), (posx,posy),15)
        

def ground_phy(space):
    body = pymunk.Body(body_type=pymunk.Body.STATIC)
    body.position = (100,300)
    #body.postion = ground_rect.center 
    shape = pymunk.Poly.create_box(body , (8000,0))
    #shape = pymunk.Circle(body,50)
    space.add(body,shape)
    return shape

# def drawball(ballsss):
#     for b in ballsss:
#         posx= int(b.body.position.x)
#         posy= int(b.body.position.y)
#         pygame.draw.circle(screen,(0,0,0),(posx,posy),50)


pygame.init()
screen = pygame.display.set_mode((800,400))
clock= pygame.time.Clock()

sky_surface=pygame.image.load('graphics/Sky.png').convert()
ground_surface= pygame.image.load('graphics/ground1.png').convert()
ground_rect= ground_surface.get_rect(topleft=(0,300))

#space-details
space = pymunk.Space()
space.gravity=(0,200)
balls=[]
joints=[]
motors = []



balls.append(falling_ball(space , 410,80))
balls.append(falling_ball(space , 380,40))
balls.append(falling_ball(space, 350, 0))
balls.append(falling_ball(space, 320,40))
balls.append(falling_ball(space , 290,80))

joints.append(String_legs(space, 0,1))
joints.append(String_legs(space, 1,2))
joints.append(String_legs(space, 2,3))
joints.append(String_legs(space, 3,4))

# motors.append(jmotors(space, 0 , 1))
# motors.append(jmotors(space, 1 , 2))
# motors.append(jmotors(space, 2 , 3))
# motors.append(jmotors(space, 3 , 4))

print(motors)
# print(joints)


# print(balls[0].body.position.x)

print(int(balls[0].body.position.x))

#balls[0].body.apply_force_at_local_point((0, 400), (int(balls[0]body.position.x), int(balls[0]body.position.y)))


# ballsss=[]
# ballsss.append(ground_phy(space))

#groundphy = ground_phy(space)



pygame 2.0.1 (SDL 2.0.14, Python 3.9.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


2021-11-30 14:42:57.478 Python[37218:7862564] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/8d/g7b_ycfj6tz940k6wlgryp240000gn/T/org.python.python.savedState


[]
410


In [2]:
#fetching coordinates - leftmost feet (x4,y4)
x4= balls[4].body.position.x 
x3= balls[3].body.position.x
x2= balls[2].body.position.x
x1= balls[1].body.position.x
x0= balls[0].body.position.x
y4=balls[4].body.position.y
y3=balls[3].body.position.y
y2=balls[2].body.position.y
y1=balls[1].body.position.y
y0=balls[0].body.position.y

inputNeurons= [x4-x3,x2-x3,x1-x3,x0-x3,y4,y3,y2,y1,y0] #relative x coordinates and y coordinates of the balls,Hence 9 inputs in total.
#285.10000000149034 - this is the y co-ordinate of ground 
#outputActions= [f0x,f0y,f3x,f4x,f0y,f1y,f3y,f4y]


In [3]:
print(inputNeurons)

[-30.0, 30.0, 60.0, 90.0, 80.0, 40.0, 0.0, 40.0, 80.0]


In [4]:
#Hidden Layers and hidden neurons

Lsa1 = [0]*900   #Layer 1 connections of S-A network (Strength of the connections)
Lsa2 = [0]*10000 #Layer 2 connections of S-A network (Strength of the connections)
Lsa2=[0]*1000    #Layer 3 connections of S-A network (Strength of the connections)
sahn1 = [0]*100  #Layer1 hidden neurons of layer 1 of S-A
sahn2=[0]*80    #Layer2 hidden neurons of layer 2 of S-A
Forces = [0]*8     #output forces F0x, F1x ....F0y,....F4y

Lar1 = [0]*800  #Layer 1 connections of A-R network (Strength of the connections)
Lar2 = [0]*10000 #Layer 2 connections of A-R network (Strength of the connections)
Lar2=[0]*100     #Layer 3 connections of A-R network (Strength of the connections)
arhn1 = [0]*100  #Layer1 hidden neurons of layer 1 of A-R
arhn2=[0]*100    #Layer2 hidden neurons of layer 2 A-R
nreward = 0  #output forces F0x, F1x ....F0y,....F4y


In [5]:
#Reward Function
def getactualReward():
    x=-1*abs(balls[2].body.position.y-100)*(balls[2].body.position.x)/100
    y= 1*(balls[2].body.position.x)
    actualReward = (x+y)/((balls[2].body.position.x/100)**2 + 1)**(0.5)
    return actualReward

#Cost Function
def costFunction(nreward):
    C=(nreward-getactualReward())**2
    return C


In [6]:
actualReward = getactualReward()
actualReward

0.0

In [8]:
# pymunk.SimpleMotor( balls[0].body,balls[2].body,  100)

relangvel=4
ground_phy(space) 
shape_filter = pymunk.ShapeFilter(group=1)
balls[0].filter = shape_filter
balls[1].filter = shape_filter
balls[2].filter = shape_filter
balls[3].filter = shape_filter
balls[4].filter = shape_filter   

timer_event = pygame.USEREVENT 
pygame.time.set_timer(timer_event, 1)
while True :
    for event in pygame.event.get():
        if event.type == pygame.QUIT :
            pygame.quit()
            exit()
            
        
        screen.blit(sky_surface,(0,0)) #building sky
        screen.blit(ground_surface,ground_rect)  #buidling ground
        draw_leg(joints)
        draw_ball(balls)
        
        
        
#         the force is calulated below using S-A network:
#             get the input neurons ( basically the positions of the balls)
#             ....
#             ...
#             ..
        
        #apply forces accordingly below:
        if int(balls[0].body.position.x) > 0:
            #print(0)
            #balls[3].body.apply_force_at_world_point((0,-400),(int(balls[2].body.position.x),int(balls[2].body.position.y)))
            balls[2].body.apply_force_at_world_point((0,-400),(int(balls[2].body.position.x),int(balls[2].body.position.y)))
            balls[1].body.apply_force_at_world_point((0,0),(int(balls[1].body.position.x),int(balls[1].body.position.y)))
            balls[3].body.apply_force_at_world_point((0,0),(int(balls[3].body.position.x),int(balls[3].body.position.y)))
            balls[0].body.apply_force_at_world_point((0,0),(int(balls[0].body.position.x),int(balls[0].body.position.y)))
            balls[4].body.apply_force_at_world_point((0,0),(int(balls[4].body.position.x),int(balls[4].body.position.y)))
            #print(balls[0].body.position)
        #drawball(ballsss)
        print(getactualReward())
        
#        network reward is calculated below using A-R network:
#              ....
#              ...
#              .
  
#         calculate actual reward and cost function below:
#              ....
#              ...
#              .


#         gradient descent stuff below to adjust Lsa and Lar: 
#              ....
#              ...
#              .
        
        



        
        
        
        space.step(0.02)
        pygame.display.update()
        clock.tick(120)
        
        

-47.88047441787352
-48.39013854614552
-48.903079150802455
-49.41852252633892
-49.93564254245787
-50.453562462421466
-50.97135477966599
-51.48804188538465
-52.002597166010304
-52.51394658381499
-53.02097079147632
-53.52250782691931
-54.01735642824658
-54.50427999995577
-54.98201125088646
-55.449257511527364
-55.90470672367535
-56.34703407933639
-56.7749092687066
-57.18700427971061
-57.582001674629964
-57.95860325361897
-58.315539001150874
-58.651576200371544
-58.96552859253135
-59.25626545449512
-59.522720466937564
-59.76390024910502
-59.97889244260586
-60.16687323601107
-60.32711423338147
-60.4589885823931
-60.56197629072665
-60.63566867213442
-60.67977187558835
-60.69410946184736
-60.67862400161286
-60.63337767834073
-60.5585518871131
-60.454445829234366
-60.321474110927625
-60.160163364157164
-59.97114791852563
-59.75516456554715
-59.513046470266644
-59.24571629982918
-58.95417865359742
-58.63951189398877
-58.30285949046621
-57.94542100015741
-57.568442816577814
-57.173208822233384
-

-35.131500303574576
-35.132856829343886
-35.133644704601544
-35.133859097899
-35.133494950201516
-35.132547038178316
-35.131010066624846
-35.1288787944688
-35.12614819992892
-35.12281369152143
-35.11887137273483
-35.114318369318596
-35.10915322923963
-35.1033764064374
-35.096990840532705
-35.09000264557717
-35.08242192173724
-35.074263704421206
-35.065549065718336
-35.05630638303479
-35.04657278937751
-35.03639581873586
-35.025835258298066
-35.014965216666084
-34.99153430058999
-34.967155581076796
-34.94554707780282
-34.92777804553574
-34.91365851501702
-34.902924591857605
-34.894926340287
-34.889770749751854
-34.887592015136136
-34.88854192370925
-34.8927790171331
-34.89795694195005
-34.90724332409245
-34.9219713503438
-34.94110973094988
-34.96445389198228
-34.992319346732124
-35.02082828105153
-35.05246562257294
-35.08815768717989
-35.12813811528184
-35.17268759095288
-35.22216090536315
-35.27694972959648
-35.337476667445436
-35.404195136650515
-35.477587787716196
-35.55816411032842


KeyboardInterrupt: 

#ball2 - center
#ball1 - first right
#ball0 - rightmost
#ball3 - first on left
#ball4 -leftmost
#all balls of length 50 units

In [ ]:
import sys

import pygame
from pygame.locals import USEREVENT, QUIT, KEYDOWN, KEYUP, K_s, K_r, K_q, K_ESCAPE, K_UP, K_DOWN, K_RIGHT, K_LEFT
from pygame.color import THECOLORS

import pymunk
from pymunk import Vec2d
import pymunk.pygame_util

class Simulator(object):

    def __init__(self):
        self.display_flags = 0
        self.display_size = (600, 600)

        self.space = pymunk.Space()
        self.space.gravity = (0.0, -1900.0)
        #self.space.damping = 0.999 # to prevent it from blowing up.

        # Pymunk physics coordinates start from the lower right-hand corner of the screen.
        self.ground_y = 100
        ground = pymunk.Segment(self.space.static_body, (5, self.ground_y), (595, self.ground_y), 1.0)
        ground.friction = 1.0
        self.space.add(ground)

        self.screen = None

        self.draw_options = None

    def reset_bodies(self):
        for body in self.space.bodies:
            if not hasattr(body, 'start_position'):
                continue
            body.position = Vec2d(body.start_position)
            body.force = 0, 0
            body.torque = 0
            body.velocity = 0, 0
            body.angular_velocity = 0
            body.angle = body.start_angle

    def draw(self):        
        self.screen.fill(THECOLORS["white"])### Clear the screen        
        self.space.debug_draw(self.draw_options)### Draw space        
        pygame.display.flip()### All done, lets flip the display

    def main(self):
        pygame.init()
        self.screen = pygame.display.set_mode(self.display_size, self.display_flags)
        width, height = self.screen.get_size()
        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)

        def to_pygame(p):            
            return int(p.x), int(-p.y+height) #Small hack to convert pymunk to pygame coordinates
        def from_pygame(p):
            return to_pygame(p)

        clock = pygame.time.Clock()
        running = True
        font = pygame.font.Font(None, 16)

        # Create the spider
        chassisXY = Vec2d(self.display_size[0]/2, self.ground_y+100)
        chWd = 70; chHt = 50
        chassisMass = 10

        legWd_a = 50; legHt_a = 5
        legWd_b = 100; legHt_b = 5
        legMass = 1
        relativeAnguVel = 0

        #---chassis
        chassis_b = pymunk.Body(chassisMass, pymunk.moment_for_box(chassisMass, (chWd, chHt)))
        chassis_b.position = chassisXY
        chassis_shape = pymunk.Poly.create_box(chassis_b, (chWd, chHt))
        chassis_shape.color = 200, 200, 200
        print("chassis position");print(chassis_b.position)

        #---first left leg a
        leftLeg_1a_body = pymunk.Body(legMass, pymunk.moment_for_box(legMass, (legWd_a, legHt_a)))
        leftLeg_1a_body.position = chassisXY - ((chWd/2)+(legWd_a/2), 0)
        leftLeg_1a_shape = pymunk.Poly.create_box(leftLeg_1a_body, (legWd_a, legHt_a))        
        leftLeg_1a_shape.color = 255, 0, 0

        #---first left leg b
        leftLeg_1b_body = pymunk.Body(legMass, pymunk.moment_for_box(legMass, (legWd_b, legHt_b)))
        leftLeg_1b_body.position = leftLeg_1a_body.position - ((legWd_a/2)+(legWd_b/2), 0)
        leftLeg_1b_shape = pymunk.Poly.create_box(leftLeg_1b_body, (legWd_b, legHt_b))        
        leftLeg_1b_shape.color = 0, 255, 0        

        #---first right leg a
        rightLeg_1a_body = pymunk.Body(legMass, pymunk.moment_for_box(legMass, (legWd_a, legHt_a)))
        rightLeg_1a_body.position = chassisXY + ((chWd/2)+(legWd_a/2), 0)
        rightLeg_1a_shape = pymunk.Poly.create_box(rightLeg_1a_body, (legWd_a, legHt_a))        
        rightLeg_1a_shape.color = 255, 0, 0        

        #---first right leg b
        rightLeg_1b_body = pymunk.Body(legMass, pymunk.moment_for_box(legMass, (legWd_b, legHt_b)))
        rightLeg_1b_body.position = rightLeg_1a_body.position + ((legWd_a/2)+(legWd_b/2), 0)
        rightLeg_1b_shape = pymunk.Poly.create_box(rightLeg_1b_body, (legWd_b, legHt_b))        
        rightLeg_1b_shape.color = 0, 255, 0     

        #---link left leg b with left leg a       
        pj_ba1left = pymunk.PinJoint(leftLeg_1b_body, leftLeg_1a_body, (legWd_b/2,0), (-legWd_a/2,0))#anchor point coordinates are wrt the body; not the space
        motor_ba1Left = pymunk.SimpleMotor(leftLeg_1b_body, leftLeg_1a_body, relativeAnguVel)
        #---link left leg a with chassis
        pj_ac1left = pymunk.PinJoint(leftLeg_1a_body, chassis_b, (legWd_a/2,0), (-chWd/2, 0))
        motor_ac1Left = pymunk.SimpleMotor(leftLeg_1a_body, chassis_b, relativeAnguVel)
        #---link right leg b with right leg a       
        pj_ba1Right = pymunk.PinJoint(rightLeg_1b_body, rightLeg_1a_body, (-legWd_b/2,0), (legWd_a/2,0))#anchor point coordinates are wrt the body; not the space
        motor_ba1Right = pymunk.SimpleMotor(rightLeg_1b_body, rightLeg_1a_body, relativeAnguVel)
        #---link right leg a with chassis
        pj_ac1Right = pymunk.PinJoint(rightLeg_1a_body, chassis_b, (-legWd_a/2,0), (chWd/2, 0))
        motor_ac1Right = pymunk.SimpleMotor(rightLeg_1a_body, chassis_b, relativeAnguVel)              

        self.space.add(chassis_b, chassis_shape) 
        self.space.add(leftLeg_1a_body, leftLeg_1a_shape, rightLeg_1a_body, rightLeg_1a_shape) 
        self.space.add(leftLeg_1b_body, leftLeg_1b_shape, rightLeg_1b_body, rightLeg_1b_shape) 
        self.space.add(pj_ba1left, motor_ba1Left, pj_ac1left, motor_ac1Left)  
        self.space.add(pj_ba1Right, motor_ba1Right, pj_ac1Right, motor_ac1Right)      

        #---prevent collisions with ShapeFilter
        shape_filter = pymunk.ShapeFilter(group=1)
        chassis_shape.filter = shape_filter
        leftLeg_1a_shape.filter = shape_filter
        rightLeg_1a_shape.filter = shape_filter
        leftLeg_1b_shape.filter = shape_filter
        rightLeg_1b_shape.filter = shape_filter        


        simulate = False
        rotationRate = 2
        while running:
            for event in pygame.event.get():
                if event.type == QUIT or (event.type == KEYDOWN and event.key in (K_q, K_ESCAPE)):
                    #running = False
                    sys.exit(0)
                elif event.type == KEYDOWN and event.key == K_s:
                    # Start/stop simulation.
                    simulate = not simulate
                elif event.type == KEYDOWN and event.key == K_r:
                    # Reset.
                    # simulate = False
                    self.reset_bodies()
                elif event.type == KEYDOWN and event.key == K_UP:
                    motor_ba1Left.rate = rotationRate
                elif event.type == KEYDOWN and event.key == K_DOWN:
                    motor_ba1Left.rate = -rotationRate
                elif event.type == KEYDOWN and event.key == K_LEFT:
                    motor_ac1Left.rate = rotationRate
                elif event.type == KEYDOWN and event.key == K_RIGHT:
                    motor_ac1Left.rate = -rotationRate                    
                elif event.type == KEYUP:
                    motor_ba1Left.rate = 0
                    motor_ac1Left.rate = 0

            self.draw()

            ### Update physics
            fps = 50
            iterations = 25
            dt = 1.0/float(fps)/float(iterations)
            if simulate:
                for x in range(iterations): # 10 iterations to get a more stable simulation
                    self.space.step(dt)

            pygame.display.flip()
            clock.tick(fps)

if __name__ == '__main__':
    sim = Simulator()
    sim.main()

In [ ]:
# 